In [32]:
##### Always import all needed libraries in the first cell
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from sklearn.svm import SVC
import pandas as pd
from string import punctuation
import os
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_predict
from sklearn import metrics

from keras.models import Sequential, load_model
from keras import layers
from keras.layers import Reshape
from keras import metrics
from keras import backend as K
from keras.utils import multi_gpu_model, CustomObjectScope
import random
import keras.backend.tensorflow_backend
import tensorflow as tf
from tqdm import tqdm
from keras.backend import clear_session

from sklearn.svm import SVC

from joblib import delayed, Parallel
np.random.seed(1) # this sets the seed so that the runs are consistent

%matplotlib inline

In [2]:
def cross_val(clf,X,y,name):
    print(name)
    y_pred = cross_val_predict(clf, X, y, cv=10)
    print(metrics.classification_report(y, y_pred))
    conf = np.array(metrics.confusion_matrix(y, y_pred))
    print(conf)
    y_probas = clf.predict_proba(X)
#     skplt.metrics.plot_roc_curve(y, y_probas, title=name+' ROC Curves', curves='each_class')
    return metrics.f1_score(y,y_pred,pos_label=1, average='binary')

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))
def flatten(l):
    return [item for sublist in l for item in sublist]

In [3]:
dftrain_banned = pd.read_csv("../Data/Generated/200_words_10M_banned.csv", delimiter=',')
dftrain_banned.insert(0, "banned", 1)

dftrain_notbanned = pd.read_csv("../Data/Generated/200_words_10M_notbanned.csv", delimiter=',')
dftrain_notbanned.insert(0, "banned", 0)

dfTest = pd.read_csv("../Data/Generated/200_words_10M_test.csv", delimiter=',')
dfTest = dfTest.sample(frac=1)

dfTest["split"] = dfTest["words"].map(lambda x: x.split(" "), na_action='ignore')
dfTest["word_cnt"] = dfTest["split"].map(lambda x: len(x), na_action='ignore')
print("Test percent lost: %.2f" % (100*len(dfTest[dfTest["word_cnt"] != 200])/ len(dfTest)))
dfTest = dfTest[dfTest["word_cnt"] == 200]



Test percent lost: 5.63


In [4]:
def group_by_subreddit(df):
    subreddit_to_comments = {}
    for index, row in df.iterrows():
        if row["subreddit"] in subreddit_to_comments:
            subreddit_to_comments[row["subreddit"]] += [row["words"]]
        else:
            subreddit_to_comments[row["subreddit"]] = [row["words"]]
    return subreddit_to_comments
    
random.seed(42)
unique_subreddits = list(dfTest["subreddit"].value_counts().keys())
random.shuffle(unique_subreddits)
SUBREDDIT_CNT = len(unique_subreddits)

s1 = int (1/3 * SUBREDDIT_CNT)
s2 = int (2/3 * SUBREDDIT_CNT)

validation_test = unique_subreddits[:s1]

threshold_test = unique_subreddits[s1:s2]

testing_test = unique_subreddits[s2:]


dfVal = dfTest[dfTest["subreddit"].isin(validation_test)]
print(dfVal["banned"].sum()/ dfVal.shape[0])


dfThresh = dfTest[dfTest["subreddit"].isin(threshold_test)]
print(dfThresh["banned"].sum()/ dfThresh.shape[0])
threshSubreddits  = group_by_subreddit(dfThresh)

dfTestingTest = dfTest[dfTest["subreddit"].isin(testing_test)]
print(dfTestingTest["banned"].sum()/ dfTestingTest.shape[0])
testingTestSubreddits  = group_by_subreddit(dfTestingTest)


0.006273352489925531
0.0050474729282013655
0.005066576393590611


In [5]:
TRAIN_BALANCE_RATIO = 20
TEST_BALANCE_RATIO = 185
TRAIN_N_COMMENTS = int(len(dftrain_banned)/3)
TEST_N_COMMENTS = int(len(dfVal)/10)

In [6]:
dfTest_banned = dfVal[dfVal["banned"]]
dfTest_notbanned = dfVal[dfVal["banned"] == False]

dfTest_balanced = pd.concat([dfTest_banned.head(n=TEST_N_COMMENTS), dfTest_notbanned.head(n=TEST_BALANCE_RATIO*TEST_N_COMMENTS)]).sample(frac=1)

In [7]:
dfTrain_banned_tmp = pd.concat([dftrain_banned.head(n=TRAIN_N_COMMENTS)]*int(TRAIN_BALANCE_RATIO))
dfTrain = pd.concat([dfTrain_banned_tmp, dftrain_notbanned.head(n=TRAIN_BALANCE_RATIO*TRAIN_N_COMMENTS)])

In [8]:
dfTrain[dfTrain["banned"]==1].shape, dfTrain[dfTrain["banned"]==0].shape

((50000, 2), (50000, 2))

In [9]:
dfTrain["split"] = dfTrain["words"].apply(lambda x: x.split(" "))
dfTrain["word_cnt"] = dfTrain["split"].apply(lambda x: len(x))
print("Train percent lost: %.2f" % (100*len(dfTrain[dfTrain["word_cnt"] != 200])/ len(dfTrain)))
dfTrain = dfTrain[dfTrain["word_cnt"]== 200]

Train percent lost: 1.00


In [10]:
dfTrain = dfTrain.sample(frac=1)
dfTrain.head(n=10)

,banned,words,split,word_cnt
587,1,progress . Because they are egotistical attent...,"[progress, ., Because, they, are, egotistical,...",200
19245,0,"living . Oh , the tears of unfathomable sadnes...","[living, ., Oh, ,, the, tears, of, unfathomabl...",200
1592,1,Plantinga . ] ( https : //www.youtube.com/watc...,"[Plantinga, ., ], (, https, :, //www.youtube.c...",200
31115,0,infantry ? I guess I just went full retarded ....,"[infantry, ?, I, guess, I, just, went, full, r...",200
2274,1,it makes me wish the world would catch on fire...,"[it, makes, me, wish, the, world, would, catch...",200
27052,0,to score pussy . Getting laid is awesome but i...,"[to, score, pussy, ., Getting, laid, is, aweso...",200
2410,1,a post about it . Good luck ! I meant mana in ...,"[a, post, about, it, ., Good, luck, !, I, mean...",200
71,1,of rules . **** *How to use AceStream/Sopcast ...,"[of, rules, ., ****, *How, to, use, AceStream/...",200
1588,1,"I did actually ! It was oddly good , haha Then...","[I, did, actually, !, It, was, oddly, good, ,,...",200
1206,1,. They want us to stop wanting to be free . gu...,"[., They, want, us, to, stop, wanting, to, be,...",200


In [14]:
with CustomObjectScope({"f1_m": f1_m, "cross_val":cross_val, "recall_m":recall_m, "precision_m":precision_m}):
    model_loaded = load_model("../Data/Cached/Model.h5")
    parallel_model = multi_gpu_model(model_loaded, gpus=8, cpu_merge=False)


In [15]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(dfTrain["words"])
y_train = dfTrain["banned"]

X_test = vectorizer.transform(dfTest_balanced["words"])
y_test = dfTest_balanced["banned"]

In [16]:
to_process = flatten(threshSubreddits.values())
X_thresh = vectorizer.transform(to_process)
# res =  parallel_model.predict(X_thresh)

# def predict_batch(examples):
#     X_thresh = vectorizer.transform(examples)
#     return parallel_model.predict(X_thresh, batch_size=32)
    
# # predict_batch(threshSubreddits["fakeid"])
# predictions = Parallel(n_jobs=-1)(delayed(predict_batch)(examples) for examples in tqdm(threshSubreddits.values()))
# # thresh_predications = {}

# for subreddit, examples in tqdm(threshSubreddits.items()):
#     X_thresh = vectorizer.transform(examples)
#     print(len(examples), len(threshSubreddits), flush=True)    
#     thresh_predications[subreddit] = model_loaded.predict(X_thresh, batch_size=len(examples))


KeyboardInterrupt: 

In [37]:
preds = []
for i in tqdm(range(int(len(to_process)/2000))):
    preds += list(parallel_model.predict(X_thresh[i*2000:(i+1)*2000]).flatten())
    print(len(preds), preds[-2:])


  1%|▏         | 1/67 [00:04<05:25,  4.93s/it]

2000 [6.2388445e-08, 2.2498864e-07]



  3%|▎         | 2/67 [00:09<05:18,  4.90s/it]

4000 [3.8185553e-06, 6.23929e-07]



  4%|▍         | 3/67 [00:14<05:13,  4.90s/it]

...

130000 [1.1676733e-07, 1.887073e-07]



 99%|█████████▊| 66/67 [05:21<00:04,  4.81s/it]

132000 [3.5820494e-07, 5.213222e-07]



100%|██████████| 67/67 [05:26<00:00,  4.82s/it]

134000 [2.46825e-06, 2.9163455e-06]


In [40]:
subreddit_to_preds = {}
i = 0
for subreddit, examples in threshSubreddits.items():
    subreddit_to_preds[subreddit] = preds[i:i+len(examples)]
    i += len(examples)
#     subreddit_keys.append([subreddit] * len(examples))


In [95]:
# for subreddit, examples in threshSubreddits.items():
#     print(len(examples), len(subreddit_to_preds[subreddit]))
def read_banned_subreddits(filename):
    """read in banned subreddits file with each line as 'r/asfasdf' """
    with open(filename, "r") as f:
        banned = set()
        for line in f:
            if line.strip() != "":
                banned.update([line.strip().split("r/")[1]])
        return banned

banned_subreddits = read_banned_subreddits("../Data/banned-subreddits.txt")
    
subreddit_to_avg = {}
for subreddit, preds in subreddit_to_preds.items():
    if preds:
        subreddit_to_avg[subreddit] = np.mean(preds)
    
banned_means = [(mean, "banned") for subreddit, mean in subreddit_to_avg.items() 
                if subreddit in banned_subreddits]
notbanned_means = [(mean, "notbanned") for subreddit, mean in subreddit_to_avg.items() 
                if subreddit not in banned_subreddits]

svm_input = pd.DataFrame(banned_means + notbanned_means, columns=["mean", "isbanned"])

svm_model = SVC(gamma="scale")

svm_model.fit([[mean] for mean in svm_input["mean"]], svm_input["isbanned"])
svm_model.__dict__

ratio = 0.17
print(sum(np.array([x for x,y in banned_means]) > ratio) / len(banned_means))
print(sum(np.array([x for x,y in notbanned_means]) < ratio) / len(notbanned_means))

# means = np.array(sorted(subreddit_to_avg.values()))


0.8333333333333334
0.9918559794256322


In [ ]:
input_dim = X_train.shape[1]  # Number of features
print(X_train.shape)
print(input_dim)

model = Sequential()


model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.5))

model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dropout(0.5))

# model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
# model.add(layers.Dropout(0.5))

# model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
# model.add(layers.Dropout(0.5))


model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
# model.add(layers.Dense(100, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [59]:
parallel_model = multi_gpu_model(model, gpus=8, cpu_merge=False)
parallel_model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc',f1_m,precision_m, recall_m])
parallel_model.summary()

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['acc',f1_m,precision_m, recall_m])
# model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dense_6_input (InputLayer)      (None, 193952)       0                                            
__________________________________________________________________________________________________
lambda_9 (Lambda)               (None, 193952)       0           dense_6_input[0][0]              
__________________________________________________________________________________________________
lambda_10 (Lambda)              (None, 193952)       0           dense_6_input[0][0]              
__________________________________________________________________________________________________
lambda_11 (Lambda)              (None, 193952)       0           dense_6_input[0][0]              
__________________________________________________________________________________________________
lambda_12 

In [60]:
history = parallel_model.fit(X_train, y_train,
                    epochs=2,
                    verbose=True,
                    validation_data=(X_test[:10000], y_test[:10000]),
                    batch_size=1024)

Train on 98995 samples, validate on 10000 samples
Epoch 1/2
98995/98995 [==============================] - 145s 1ms/step - loss: 0.2650 - acc: 0.8764 - f1_m: 0.7945 - precision_m: 0.9613 - recall_m: 0.7653 - val_loss: 0.0439 - val_acc: 0.9907 - val_f1_m: 0.2823 - val_precision_m: 0.2540 - val_recall_m: 0.3742
Epoch 2/2
98995/98995 [==============================] - 140s 1ms/step - loss: 0.0068 - acc: 0.9988 - f1_m: 0.9987 - precision_m: 0.9979 - recall_m: 0.9996 - val_loss: 0.0515 - val_acc: 0.9911 - val_f1_m: 0.3076 - val_precision_m: 0.2823 - val_recall_m: 0.4510


In [127]:
parallel_model.save("../Data/Cached/Model.h5")

In [129]:
with CustomObjectScope({"f1_m": f1_m, "cross_val":cross_val, "recall_m":recall_m, "precision_m":precision_m}):
    model_loaded = load_model("../Data/Cached/Model.h5")

In [ ]:
# X_thresh = vectorizer.transform(flatten(threshSubreddits.values()))

def predict_batch(examples):
    X_thresh = vectorizer.transform(examples)
    return parallel_model.predict(X_thresh, batch_size=256)
    
    
predictions = Parallel(n_jobs=-1)(delayed(predict_batch)(examples) for examples in tqdm(threshSubreddits.values()))

# for subreddit, examples in threshSubreddits.items():
#     X_thresh = vectorizer.transform(examples)
#     print(len(examples), flush=True)
#     thresh_predications[subreddit] = parallel_model.predict(X_thresh, batch_size=len(examples))
    

In [135]:
parallel_model.__dict__

{'_uses_inputs_arg': True,
 'inputs': [<tf.Tensor 'dense_6_input:0' shape=(?, 193952) dtype=float32>],
 'outputs': [<tf.Tensor 'dense_10_1/concat:0' shape=(?, 1) dtype=float32>],
 'name': 'model_2',
 'trainable': True,
 '_is_compiled': True,
 '_expects_training_arg': False,
 '_initial_weights': None,
 'supports_masking': False,
 'optimizer': <keras.optimizers.Adam at 0x2b5b6e8ec278>,
 '_updates': [],
 '_losses': [],
 '_per_input_losses': {},
 '_per_input_updates': {},
 '_layers': [<keras.engine.input_layer.InputLayer at 0x2b5e26dce668>,
 '_outbound_nodes': [],
 '_inbound_nodes': [<keras.engine.base_layer.Node at 0x2b5e26c66278>],
 '_compute_previous_mask': True,
 '_built': True,
 '_is_graph_network': True,
 '_input_layers': [<keras.engine.input_layer.InputLayer at 0x2b5e26dce668>],
 '_output_layers': [<keras.layers.merge.Concatenate at 0x2b5e26c66048>],
 '_input_coordinates': [(<keras.engine.input_layer.InputLayer at 0x2b5e26dce668>,
   0,
   0)],
 '_output_coordinates': [(<keras.layer

In [132]:
prediction = model_loaded.predict(X_thresh, use_multiprocessing=True)


TypeError: predict() got an unexpected keyword argument 'use_multiprocessing'

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [ ]:
plot_history(history)